# Set-up

# UniRel

- Model:
    - Github: https://github.com/wtangdev/unirel
    - Paper: https://arxiv.org/abs/2211.09039

In [49]:
'''from google.cloud import storage

def download_file_from_gcs(source_blob_name, destination_file_name):
    """Downloads a file from Google Cloud Storage to the local directory."""
    storage_client = storage.Client()

    # Specify the GCS bucket and file path
    bucket_name = "msca-sp23-bucket"
    blob = storage_client.bucket(bucket_name).blob(source_blob_name)

    # Download the file
    blob.download_to_filename(destination_file_name)

# Call the function to download the file
source_blob_name = "capstone_data/UniRel.zip"
destination_file_name = "/home/anthony/projects/capstone_runtime/UniRel.zip"

download_file_from_gcs(source_blob_name, destination_file_name)'''

'from google.cloud import storage\n\ndef download_file_from_gcs(source_blob_name, destination_file_name):\n    """Downloads a file from Google Cloud Storage to the local directory."""\n    storage_client = storage.Client()\n\n    # Specify the GCS bucket and file path\n    bucket_name = "msca-sp23-bucket"\n    blob = storage_client.bucket(bucket_name).blob(source_blob_name)\n\n    # Download the file\n    blob.download_to_filename(destination_file_name)\n\n# Call the function to download the file\nsource_blob_name = "capstone_data/UniRel.zip"\ndestination_file_name = "/home/anthony/projects/capstone_runtime/UniRel.zip"\n\ndownload_file_from_gcs(source_blob_name, destination_file_name)'

In [50]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

import warnings
warnings.filterwarnings("ignore")

In [51]:
from predict import UniRel

model_path = '/home/anthony/projects/capstone_runtime/UniRel'
unirel = UniRel(model_path, dataset_name="nyt")

## Test on sentences

In [52]:
text = 'Bryan Kohberger was accused of killing four Idaho college students in an early morning attack last year, according to new court documents.'
print(unirel.predict(text))

[[('Bryan Kohberger', '/people/person/place_lived', 'Idaho')]]


In [53]:
text = 'Bryan Kohberger was accused of killing four Idaho college students in an early morning'
print(unirel.predict(text))

[[('Bryan Kohberger', '/people/person/place_lived', 'Idaho')]]


In [54]:
text = 'Bryan Kohberger was accused of killing four college students'
print(unirel.predict(text))

[[]]


In [55]:
text = 'Timothy Donald Cook is an American business executive who has been the chief executive officer of Apple Inc. since 2011. '
print(unirel.predict(text))

[[('Timothy Donald Cook', '/business/person/company', 'Apple Inc.')]]


In [4]:
print(unirel.predict("In perhaps the most ambitious Mekong cruise attempt, Impulse Tourism, an operator based in Chiang Mai, Thailand, is organizing an expedition starting in November in Jinghong, a small city in the Yunnan province in China."))

[[('China', '/location/location/contains', 'Jinghong'), ('Yunnan', '/location/administrative_division/country', 'China'), ('China', '/location/country/administrative_divisions', 'Yunnan'), ('Thailand', '/location/location/contains', 'Chiang Mai'), ('China', '/location/location/contains', 'Yunnan')]]


/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [6]:
input_text = 'Sam commited a robbery last week.'
print(unirel.predict(input_text))

[[]]


In [8]:
input_text = 'Sam robbed the Chase Bank at 60611 last week.'
print(unirel.predict(input_text))

[[('Sam', '/location/location/contains', 'Chase Bank')]]


In [9]:
input_text = 'Sam robbed Peter last week.'
print(unirel.predict(input_text))

[[('Peter', '/people/deceased_person/place_of_death', 'Sam')]]


In [10]:
input_text = 'Diamond merchant Nirav Modi, accused in the Punjab National Bank (PNB) fraud and money laundering case, suffered another setback in his legal battle against his extradition to India.'
print(unirel.predict(input_text))

[[('Nirav Modi', '/people/person/nationality', 'India')]]


In [11]:
input_text = 'Diamond merchant Nirav Modi, accused in the Punjab National Bank (PNB) fraud and money laundering case, suffered another setback in his legal battle'
print(unirel.predict(input_text))

[[('Nirav Modi', '/business/person/company', 'Punjab National Bank')]]


In [14]:
input_text = 'Nirav Modi was accused in the fraud and money laundering case'
print(unirel.predict(input_text))

[[]]


## Test sentences Meeting

In [56]:
input_text = 'On April 21, Liu Ti, former deputy general manager of the Shanghai Stock Exchange, has been placed under investigation on suspicion of serious violations of duties, according to China Daily.'
print(unirel.predict(input_text))

[[('Liu Ti', '/people/person/nationality', 'China')]]


In [57]:
input_text = 'Liu Ti, former deputy general manager of the Shanghai Stock Exchange, has been placed under investigation on suspicion of serious violations of duties'
print(unirel.predict(input_text))

[[('Liu Ti', '/business/person/company', 'Shanghai Stock Exchange')]]


In [58]:
input_text = 'Liu Ti, former deputy general manager, has been placed under investigation on suspicion of serious violations of duties'
print(unirel.predict(input_text))

[[]]


# REBEL

In [1]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

# We need to use the tokenizer manually since we need special tokens.
extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor("Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic", return_tensors=True, return_text=False)[0]["generated_token_ids"]])

print(extracted_text[0])

# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)

2023-06-03 17:45:04.087055: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 17:45:05.074911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<s><triplet> Punta Cana <subj> La Altagracia Province <obj> located in the administrative territorial entity <subj> Dominican Republic <obj> country <triplet> Higuey <subj> La Altagracia Province <obj> located in the administrative territorial entity <subj> Dominican Republic <obj> country <triplet> La Altagracia Province <subj> Dominican Republic <obj> country <triplet> Dominican Republic <subj> La Altagracia Province <obj> contains administrative territorial entity</s>
[{'head': 'Punta Cana', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Higuey', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'La Altagracia Province', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Dominican Republic', 'type': 'contains administrative 

In [12]:
# test sentences
raw_text_1 = 'Kevin Parkourana, 31, of San Jose was in custody in connection with the attacks Thursday, police from San Jose and the adjacent city of Milpitas announced Friday.'
# raw_text_1 = 'Kevin Parkourana, 31, of San Jose was in custody in connection with the attacks Thursday'
raw_text_2 = 'Luca Visentini, 53, charged but conditionally released, is the secretary general of the International Trade Union Confederation (ITUC), which brings together 332 unions and claims 200 million members in 163 countries.'

In [13]:
def predict(text):
    extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(text, return_tensors=True, return_text=False)[0]["generated_token_ids"]])
    extracted_triplets = extract_triplets(extracted_text[0])
    return extracted_triplets

predict(raw_text_1)

[{'head': 'San Jose', 'type': 'shares border with', 'tail': 'Milpitas'},
 {'head': 'Milpitas', 'type': 'shares border with', 'tail': 'San Jose'}]

In [14]:
predict(raw_text_2)

[{'head': 'Luca Visentini',
  'type': 'member of',
  'tail': 'International Trade Union Confederation'}]

In [23]:
import spacy
nlp = spacy.load("en_core_web_md")
import spacy_component

nlp.add_pipe("rebel", after="senter", config={
    'device':-1, # Number of the GPU, -1 if want to use CPU
    'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
    )

In [37]:
raw_text_3 = 'Kevin Parkourana was in custody in connection with the attacks Thursday, police announced Friday.'
raw_text_4 = 'A short time later, the suspect struck a person on a motorized scooter'
raw_text_5 = 'agents and police served a search warrant at an east Atlanta home and arrested Marlon Kautz and Adele Maclean, both from Atlanta'

In [33]:
input_sentence = raw_text_3
                 
doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(0, 9): {'relation': 'participant in', 'head_span': Kevin Parkourana, 'tail_span': attacks Thursday}
(9, 0): {'relation': 'participant', 'head_span': attacks Thursday, 'tail_span': Kevin Parkourana}


In [36]:
input_sentence = raw_text_4

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(12, 7): {'relation': 'significant event', 'head_span': motorized scooter, 'tail_span': struck a person on a motorized scooter}


In [39]:
#input_sentence = raw_text_5
input_sentence = 'Agents and police served a search warrant and arrested Marlon Kautz and Adele Maclean'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(9, 12): {'relation': 'spouse', 'head_span': Marlon Kautz, 'tail_span': Adele Maclean}
(12, 9): {'relation': 'spouse', 'head_span': Adele Maclean, 'tail_span': Marlon Kautz}


In [43]:
input_sentence = 'Bryan Kohberger was accused of killing four Idaho college students in an early morning attack last year, according to new court documents.'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(14, 7): {'relation': 'located in the administrative territorial entity', 'head_span': attack last year, 'tail_span': Idaho}


In [47]:
input_sentence = 'Bryan Kohberger was accused of killing four Idaho college students in an early morning'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(0, 7): {'relation': 'place of birth', 'head_span': Bryan Kohberger, 'tail_span': Idaho}


In [42]:
input_sentence = 'Bryan Kohberger was accused of killing four college students'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(0, 5): {'relation': 'significant event', 'head_span': Bryan Kohberger, 'tail_span': killing four college students}
(5, 0): {'relation': 'participant', 'head_span': killing four college students, 'tail_span': Bryan Kohberger}


In [59]:
input_sentence = 'Liu Ti, former deputy general manager, has been placed under investigation on suspicion of serious violations of duties'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(0, 4): {'relation': 'position held', 'head_span': Liu Ti, 'tail_span': deputy general manager}


In [61]:
input_sentence = 'Liu Ti has been placed under investigation on suspicion of serious violations of duties'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

In [62]:
input_sentence = 'Liu Ti has arrested under investigation on suspicion of serious violations of duties'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

In [63]:
input_sentence = 'Sam commited a robbery last week.'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(3, 0): {'relation': 'participant', 'head_span': robbery last week, 'tail_span': Sam}
(0, 3): {'relation': 'participant in', 'head_span': Sam, 'tail_span': robbery last week}


In [68]:
input_sentence = 'Peter Wang is arrested of paying a prostitute last week.'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(5, 0): {'relation': 'participant', 'head_span': paying a prostitute, 'tail_span': Peter Wang}


In [70]:
input_sentence = 'Peter Wang is charged with mutiple crimes.'

doc = nlp(input_sentence)
doc_list = nlp.pipe([input_sentence])
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

(0, 5): {'relation': 'instance of', 'head_span': Peter Wang, 'tail_span': mutiple crimes}
